# Launch Site Location Analysis With Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

**Here in this notebook I am going to perform the visual Analytics of Launch Site Location With Folium**

# Objectives
1. Launch Site Analysis With Folium.
2. EDA

![spaceX Data](https://www.teslarati.com/wp-content/uploads/2020/04/Falcon-Heavy-Demo-Feb-2018-SpaceX-1-crop-2048x956.jpg)

## Import Necessary Libraries

In [2]:
import pandas as pd
import folium

In [35]:
from folium.features import DivIcon
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition

**Read CSV file**

In [3]:
spacex = pd.read_csv('../Data/dataset_part_2.csv')
spacex.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


Lets check latitude and longitude for each site.

In [16]:
spacex[['Latitude','Longitude']][0:10]

,Latitude,Longitude
0,28.561857,-80.577366
1,28.561857,-80.577366
2,28.561857,-80.577366
3,34.632093,-120.610829
4,28.561857,-80.577366
5,28.561857,-80.577366
6,28.561857,-80.577366
7,28.561857,-80.577366
8,28.561857,-80.577366
9,28.561857,-80.577366


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

Lets Create <code>Map</code> Instance with location of Bombay Stock Exchange Building

In [7]:
# Start location is Bombay Stock Exchange Building
stock_coordinate = [18.929863, 72.833427]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

As you can see the location is not clearly visible.
Add Circle and Marker on the location so that it will be easily visible.

In [11]:
# Create a blue circle at Bombay Stock Exchange Building with a popup label showing its name
circle = folium.Circle(stock_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Bombay Stock Exchange Building'))
# Create a blue circle at Bombay Stock Exchange Building with a icon showing its name
marker = folium.map.Marker(
    stock_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(30,30),
        icon_anchor=(0,0),
        html='<div style="font-size: 13; color:#d35400;"><b>%s</b></div>' % 'Mumbai BSE',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Map All <code>LaunchSite</code> With the location So that I can Show them all on the map.

**Mapped All LaunchSites with their Location**

In [17]:
spacex_df = spacex[['LaunchSite','Longitude','Latitude','Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


# Show all Location In Map

After Setting up the LaunchSite locations on the map need to explore the map so that i can perform the better analysis on the Launchsite.

In [26]:
for index,records in launch_sites_df.iterrows():
    print(f"{index} ---> {records['LaunchSite']} ---> {records['Latitude']} ---> {records['Longitude']}")

0 ---> CCSFS SLC 40 ---> 28.5618571 ---> -80.577366
1 ---> KSC LC 39A ---> 28.6080585 ---> -80.6039558
2 ---> VAFB SLC 4E ---> 34.632093 ---> -120.610829


In [25]:
# Start location is Bombay Stock Exchange Building
stock_coordinate = [18.929863, 72.833427]
site_map = folium.Map(location=nasa_coordinate, zoom_start=20)

for index,records in launch_sites_df.iterrows():
    
    # Add Circle for Each Location      
    folium.Circle(location=[records['Latitude'],records['Longitude']], radius=1000,fill=True).add_child(folium.Popup(f"{records['LaunchSite']}")).add_to(site_map)
    
    # Add Marker For Each LaunchSite Location
    folium.Marker(location=[records['Latitude'],records['Longitude']],
                  icon=DivIcon(icon_size=(30,30),
                  icon_anchor=(0,0),
                  html='<div style="font-size: 15; color:#d35400;"><b>%s</b></div>' % records['LaunchSite'], )).add_to(site_map)
    
site_map

#### Now its Time to Introduce the <code>Class</code> On the Map.

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not.

### By Adding <code>Class</code> We can see the Success and Filure Mark on the LaunchSite.

Instantiate The <code>MarkerCluster</code>.

In [30]:
marker_cluster = MarkerCluster()

Add one more column in the DataFrame with name mark_color and store 'red' if class is 0 and 'green' if class is 1.

In [31]:
spacex['marker_color'] = spacex['Class'].apply(lambda x: 'green' if x==1 else 'red')

In [34]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, record in spacex.iterrows():
    marker = folium.Marker(location=[record['Latitude'],record['Longitude']],
                  icon=folium.Icon(color='white', icon_color=record['marker_color'],
                  icon_anchor=(0,0),
                  html='<div style="font-size: 15; color:#d35400;"><b>%s</b></div>' % record['LaunchSite'], ))
    marker_cluster.add_child(marker)

site_map

# Let's Explore How to Calculate the Distances With Folium.

I will Add <code>MousePosition</code> On the map to add the Latitude and Longitude on the mouse postion.

**Now You can See the Lat and Long on the Map when you move the cursor Values also gets changed.**

In [40]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 4);};"
mouse_position = MousePosition(
    position='bottomright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

You can calculate the distance between two points on the map based on their Lat and Long values using the following method:

# Calculate the Distance between two Locations.

**Here I have calculated the distance with the help of python function.**

# How to Calculate the Distance?

1. The simple way of calculating a distance between two points is using the Pythagorean formula to calculate the hypotenuse of a triangle (A² + B² = C²). This is known as the Euclidean distance.

2. Above Point Not applicable in Geography since the distance between lines of latitude and longitude are not equal distances apart. As you get closer to the equator, lines of latitude get further apart. If you use some kind of simple triangulation equation, it may measure distance accurately in one location and terribly wrong in the other, because of the curvature of the Earth.

**The Haversine Formula**

The distance using the curvature of the Earth is incorporated in the Haversine formula, which uses trigonometry to allow for the curvature of the earth. When you’re finding the distance between 2 places on earth (as the crow flies), a straight line is really an arc.

In [42]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1:float, lon1:float, lat2:float, lon2:float)->float:
    
    # approximate radius of earth in km
    R = 6373.0
    
    # Convert the points into Radians    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    # Haversine Formula    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Distance To Coastline

In [43]:
#Work out distance to coastline
coordinates = [
    [28.56342, -80.57674],
    [28.56342, -80.56756]]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
distance = calculate_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])
distance_circle = folium.Marker(
    [28.56342, -80.56794],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )
site_map.add_child(distance_circle)
site_map

# Distance to Highway

In [44]:
#Distance to Highway
coordinates = [
    [28.56342, -80.57674],
    [28.411780, -80.820630]]

lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
distance = calculate_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])
distance_circle = folium.Marker(
    [28.411780, -80.820630],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#252526;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )
site_map.add_child(distance_circle)
site_map

# How to Create PolyLine in Folium

**Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate**

In [45]:
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [ ]:
#